In [ ]:
from binance.spot import Spot
import pandas as pd
import talib
import time
import datetime

def get_candles(client, symbol, interval, limit):
    # Get the candles data for the specified symbol, period, and interval
    candles = client.klines(
        symbol=symbol,
        interval=interval,
        limit=limit
    )

    # Create a Pandas DataFrame from the candles data
    df = pd.DataFrame(candles, columns=[
        'open_time',
        'open',
        'high',
        'low',
        'close',
        'volume',
        'close_time',
        'quote_asset_volume',
        'number_of_trades',
        'taker_buy_base_asset_volume',
        'taker_buy_quote_asset_volume',
        'ignore'
    ])

    # Convert the column values to numerical values
    df = df.apply(pd.to_numeric)

    # Return the Pandas DataFrame
    return df

def get_rsi(df, length):
    # Calculate the RSI
    rsi = talib.RSI(df['close'], timeperiod=length)

    # Create a Pandas DataFrame with the RSI values
    rsi_df = pd.DataFrame(rsi, columns=['RSI'])

    # Return the RSI DataFrame
    return rsi_df

def get_ema(df, length):
    # Calculate the RSI
    ema = talib.EMA(df['close'], timeperiod=length)

    # Create a Pandas DataFrame with the RSI values
    ema_df = pd.DataFrame(ema, columns=['EMA'])

    # Return the RSI DataFrame
    return ema_df

def get_sar(df, length):
    # Calculate the RSI
    sar = talib.SAR(df['high'],df['low'], acceleration=0.02, maximum=0.2)

    # Create a Pandas DataFrame with the RSI values
    sar_df = pd.DataFrame(sar, columns=['SAR'])

    # Return the RSI DataFrame
    return sar_df

# Get server timestamp
#print(client.time())

# API key/secret are required for user data endpoints
client = Spot(api_key=''
            ,api_secret='')




In [ ]:
# Get the candles data
df = get_candles(client, "BTCUSDT", "1m", limit=100)

In [ ]:
df.head()

In [ ]:


df["EMA_9"] = talib.EMA(df["close"], timeperiod=9)
df["EMA_20"] = talib.EMA(df["close"], timeperiod=20)
df["RSI"] = talib.RSI(df['close'], timeperiod=14)
df["SAR"] = talib.SAR(df['high'],df['low'], acceleration=0.02, maximum=0.2)
df["time"] = pd.to_datetime(df['open_time'], unit='ms') #IMPORTANTE!! EL TIEMPO ES UTC

In [ ]:
df.head()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Candlestick(x=df['time'],
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        name="candles"
                    ),
    secondary_y=False,
)

# Add traces
fig.add_trace(
    go.Scatter(
                        x=df['time'],
                        y=df['EMA_9'],
                        name="EMA_9"
                    ),
    secondary_y=False,
)

# Add traces
fig.add_trace(
    go.Scatter(
                        x=df['time'],
                        y=df['EMA_20'],
                        name="EMA_20"
                    ),
    secondary_y=False,
)
# Add traces
fig.add_trace(
    go.Scatter(
                        x=df['time'],
                        y=df['SAR'],
                        name="SAR",
                        mode='markers',
                    ),
    secondary_y=False,
)

# Add traces
fig.add_trace(
    go.Scatter(
                        x=df['time'],
                        y=df['RSI'],
                        name="RSI"
                    ),
    secondary_y=True,
)

In [ ]:

rsi = df['RSI'].iloc[-1]
ema_nine = df['EMA_9'].iloc[-1]
ema_twenty = df['EMA_20'].iloc[-1]
sar = df['SAR'].iloc[-1]

print(f"RSI: {rsi}")
print(f"EMA_9: {ema_nine}")
print(f"EMA_20: {ema_twenty}")
print(f"SAR: {sar}")
